# Osyris 2.0 prototype

### New features

- More Pythonic API
- Runtime physical units using pint
- Arithmetic operations between data arrays
- More generic interface for plots (easier to use custom matplotlib options)
- Cleaner data loader

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

## Loading a Ramses output

We load a data output from a star formation simulation

In [ ]:
data = osyris.load(71, scale="au")

## The `data` object

The `data` object has a `__str__` representation, which means you can get information on its content by just using the variable name:

In [ ]:
data

The `data` object aims to behave very similarly to a Python `dict`

In [ ]:
data["density"]

In [ ]:
data.keys()

In [ ]:
data.values()

Additional metadata is stored under the `meta` property

In [ ]:
data.meta

## The `Array` object

Each entry in the `data` dictionary is an `Array` object:

In [ ]:
type(data["density"])

In [ ]:
data["density"]

An `Array` can basically be thought of as a `numpy` array with a physical unit: 

In [ ]:
data["density"].values

In [ ]:
data["density"].unit

Operations you would normally perform on a numpy array, such as slicing, are supported:

In [ ]:
data["density"][101]

Using `numpy`'s `array_function` and `array_ufunc` protocols, `Array` also supports most `numpy` operations, e.g.

In [ ]:
np.log10(data["density"])

In [ ]:
np.sum(data["density"])

### Example: find system center

In [ ]:
ind = np.argmax(data["density"])
center = data["xyz"][ind.values]
center

## Array arithmetic and units

Units are automatically handled, and conversions performed, when performing arithmetic on arrays.
For instance, we want to compute a new quantity which represents the mass inside each cell.

In [ ]:
data["density"]

In [ ]:
data["dx"]

In [ ]:
data["mass"] = data["density"] * (data["dx"] * data["dx"] * data["dx"])
data["mass"]

Conversion between `au` and `cm` is automatically handled by first converting both operands to their base units, before performing the operation.

This helps to free mental capacity and allows the user to focus on what is important: doing science.

### Manual unit conversions

Sometimes, it is useful to convert from CGS units to other base units:

In [ ]:
data["mass"].to("msun")
data["mass"]

### Units also provide safety

In [ ]:
data["density"] + data["mass"]

### Automatic broadcast

Arrays can either represent a scalar quantity (e.g. density) or a vector quantity (e.g. velocity).
When performing arithmetic that involves both scalars and vectors, an automatic broadcast mechanism handles the operations:

In [ ]:
data["momentum"] = data["density"] * data["velocity"]
data["momentum"]

### Operations with floats

Operations with floats are also supported:

In [ ]:
data["density"] * 1.0e5

## Plotting

The basic functionality of Osyris 1.0 has been implemented in version 2.0.

### 2D histogram of density vs magnetic field

By default, the `histogram` function will show a binned count of cells

In [ ]:
osyris.plot.histogram(data["density"], data["B_field"],
                      norm="log", loglog=True)

Additional components can be overlayed using `layers`:

In [ ]:
osyris.plot.histogram(data["density"], data["B_field"],
                      {"data": data["mass"], "norm": "log"},
                      {"data": data["level"], "operation": "mean", "mode": "contour", "colors": "k"},
                      loglog=True)

### Simple cut plane

Create a 2D gas density slice 100 au wide through the `z` plane, with velocity vectors overlayed as arrows, once agains using `layers`:

In [ ]:
osyris.plane({"data": data["density"], "norm": "log"},
             {"data": data["velocity"], "mode": "vec"},
             dx=100,
             origin=center,
             direction="z")

You can add as many layers as you like

In [ ]:
osyris.plane({"data": data["density"], "norm": "log"},
             {"data": data["velocity"], "mode": "vec"},
             {"data": data["level"], "mode": "contour", "colors": "red"},
             {"data": data["B_field"], "mode": "stream", "color": "gray"},
             dx=100,
             origin=center,
             direction="z")

## Selective loading

In Osyris 1.0, we could load only a subset of the domain by selecting a region around the center.
This mechanism has been generalized in version 2.0 to accept a custom function to perform the selection.

As an example, to load all the cells with density > 1.0e-13 g/cm^3, we use a selection criterion

In [ ]:
data2 = osyris.load(71, scale="au", select={"density": lambda x : x > 1.0e-13})

In [ ]:
osyris.plot.histogram(data2["density"], data2["B_field"],
                      norm="log", loglog=True)

Multiple selection criteria are ANDed:

In [ ]:
data2 = osyris.load(71, scale="au", select={"density": lambda x : x > 1.0e-13,
                                            "xyz_x": lambda x : abs(x - 5517.) < 50})

In [ ]:
osyris.plane({"data": data2["density"], "norm": "log"},
             dx=200,
             origin=center,
             direction='z')

## Future work

### Was in 1.0, not yet in 2.0

- Particles and sink particles loaders
- Column density plots
- 

### Future additions/requirements

- Lazy and distributed loading (probably use [Dask](https://dask.org/)) for large datasets
- Automatic resampling on figure zoom (useful for cut planes, also useful for histograms?)
- Full operations support for `Array` (currently missing a number of operations)


## More plots

### 1. Plane at an arbitrary angle

In [ ]:
osyris.plane({"data": data["density"], "norm": "log"},
             {"data": data["velocity"], "mode": "vec", "color": data["velocity"]},
             dx=100,
             origin=center,
             direction=[-1, 1, 1])

### 2. Automatic “top/side” slice orientation according to angular momentum

Create a 2D slice of the logarithm of density 50 au wide through the data using automatic orientation based on the angular momentum in the data. This is useful for looking at disks. Use the `"top"` direction for the slice to view the disk from above

In [ ]:
osyris.plane({"data": data["density"], "norm": "log"},
             {"data": data["velocity"], "mode": "vec"},
             dx=50,
             origin=center,
             direction="top")

Use the `direction="side"` for the slice to view the disk from the side

In [ ]:
osyris.plane({"data": data["density"], "norm": "log"},
             {"data": data["velocity"], "mode": "vec"},
             dx=50,
             origin=center,
             direction="side")

### 3. Embedding plots in existing matplotlib axes

In this example, we create two subplot axes with `matplotlib`.

Next, we plot in the left panel the log of density as a coloured slice with velocity vectors.
The minimum and maximum of $\log(\rho)$ is forced to `-14` and `-9`.
We give the `plane` call the axes to use via the `ax` argument.
Next, we overlay some custom chosen density contours with different line styles and colours.

In the right panel, we plot a plane of temperature and overlay some lightgray contours showing the AMR levels.
We specify only integer contour levels.

In [ ]:
# Create figure
fig = plt.figure(figsize=(8, 3))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
plt.subplots_adjust(wspace=0.5)

# Define region to plot
dx = 15.0

# Left plot: coloured density slice with overlayed contours
osyris.plane({"data": data["density"], "norm": "log",
                               "vmin": 1.0e-14,
                               "vmax": 1.0e-9},
             {"data": data["velocity"], "mode": "vec"},
             {"data": data["density"], "mode": "contour",
                  "levels": [1.0e-12,1.0e-11,1.0e-9],
                  "colors": ('yellow','k',"lime"),
                  "linewidths": [2,5,2],
                  "linestyles": ["solid","dashed","solid"],
                  "cmap": None,
                  "labels": False},
             dx=dx,
             origin=center,
             direction="z", ax=ax1)

osyris.plane({"data": data["temperature"], "norm": "log", "mode": "contourf",
              "levels": np.logspace(1.4, 3, 21), "cmap": "hot"},
             {"data": data["level"], "mode": "contour", "colors": "w", "levels": [14, 15, 16]},
             dx=dx,
             origin=center,
             direction="z", ax=ax2)

### 4. Plot only a subset of cells belonging to a disk

In this example, we select cells according to their density and plot only those. This is done by creating a new field and using the `numpy` `where` function. To combine more than one selection criteria, use the `logical_and` `numpy` function.

This is useful for plotting disks around protostars, for example. Here we select the cells with a density in the range $-12.5 < \log(\rho) < -11.0$. After plotting the disk, we use 2 different methods to compute the disk mass.

In [ ]:
data["disk_density"] = osyris.Array(np.ma.masked_where(np.logical_or(
                                 data["density"].values < 3.0e-13,
                                 data["density"].values > 1.0e-11),
                                                       data["density"].values),
                                    unit=data["density"].unit)
data

In [ ]:
osyris.plane(data["disk_density"], dx=50, norm="log", origin=center, mode="image")

In [ ]:
np.sum(data["disk_density"]*(data["dx"]*data["dx"]*data["dx"])).to("msun")

### 5. Difference between two snapshots

Here, we want to make a map of the difference in density between two snapshots. Because we do not necessarily have the same number of cells at the same place, we first have to make uniform 2D maps using the `plot_slice` function, which we can then directly compare.

This is done by calling `plot_slice` with the arguments `plot=False` to avoid making a plot, and `copy=True` to return the data to a variable.

For this to make sense, the two outputs have to be centered around the same point: `[0.5, 0.5, 0.5]` (which is the default).

In [ ]:
# Read data from a later snapshot
data2 = osyris.load(201, scale="au")

In [ ]:
# Extract log(density) slices by copying data into structures
plane1 = osyris.plane({"data": data["density"], "norm": "log"},
             dx=100,
             origin=center,
             direction="z", plot=False)

plane2 = osyris.plane({"data": data2["density"], "norm": "log"},
             dx=100,
             origin=center,
             direction="z",
                     plot=False)

# Get coordinates
x = plane1.x
y = plane1.y

# Density difference
rho1 = np.log10(plane1.layers[0]["data"])
rho2 = np.log10(plane2.layers[0]["data"])
diff = (rho1-rho2)/rho2

# Create figure
fig, ax = plt.subplots()
im = ax.contourf(x, y , diff, cmap='RdBu',
                   levels=np.linspace(-0.12,0.12,31))
ax.set_aspect("equal")
cb = plt.colorbar(im, ax=ax)
cb.ax.set_ylabel("Relative difference")

### 6. Slice above the origin

We want to plot a slice of density but through a point which is 5 AU above the centre of the domain, defined as the cell with the highest density. This is done by setting the `origin` coordinate to `[0, 0, 5]`

In [ ]:
origin = center.copy()
origin.values[-1] += 5
osyris.plane({"data": data["density"], "norm": "log"},
             {"data": data["velocity"], "mode": "vec"},
             dx=100,
             origin=origin,
             direction="z")